In [1]:
import torch
from torch import nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader

import torchvision
from torchvision import models, transforms

import matplotlib.pyplot as plt
import numpy as np
import scipy.io
import os
import random
import cv2
import copy
import time
from tqdm import tqdm

plt.ion()

### Only runs once:

In [ ]:
# root = os.path.abspath('jpg')
# labels = scipy.io.loadmat('imagelabels.mat')['labels'].squeeze()

# for cls in range(102):
#     cls_folder = os.path.join(root, f'{cls}')
#     os.makedirs(cls_folder, exist_ok=True)

#     indices = np.argwhere(labels==cls+1).squeeze()
    
#     for idx in indices:
#         s_idx = str(idx+1)

#         while len(s_idx) != 5:
#             s_idx = '0'+s_idx

#         file = os.path.join(root, f'image_{s_idx}.jpg')
#         try:
#             os.rename(file, os.path.join(cls_folder, f'{s_idx}.jpg'))
#         except OSError as e:
#             print(e)


In [ ]:
labels = scipy.io.loadmat('imagelabels.mat')['labels'].squeeze()

In [ ]:
def seed_all(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.random.manual_seed(seed)
    
def data_map(path):
    dmap = {}
    for root, dirs, files in os.walk(path):
        if files:
            k = os.path.split(root)[-1]
            k = int(k.split('_')[0])
            dmap[k] = np.array([os.path.join(root, f) for f in files])
            
    return dmap
        
def train_val_test_split(dmap, p_val=0.25, p_test=0.25):
    train, val, test = {}, {}, {}
    
    for cls, samples in dmap.items():
        num_val = int(p_val * len(samples))
        num_test = int(p_test * len(samples))
        
        data = samples.copy()
        np.random.shuffle(data)
        
        val[cls] = data[:num_val].copy()
        test[cls] = data[num_val:num_val+num_test].copy()
        train[cls] = data[num_val+num_test:].copy()
    
    return train, val, test

def get_mean_std(dmap):
    trns = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((256, 256)),
    ])
    
    n_samples = sum([len(v) for v in dmap.values()])
    images = torch.zeros((n_samples, 3, 256, 256), dtype=torch.float32)
    
    idx = 0
    for samples in dmap.values():
        for path in samples:
            print(f'\r{idx+1}/{n_samples}', end='')
            img = cv2.imread(path)[:,:,::-1].copy()
            img = trns(img)
            images[idx] = img
            idx += 1
    
    print()
    
    images = images.transpose(1, 0).reshape(3, -1)
    return {'mean': images.mean(dim=-1), 'std': images.std(dim=-1)}

seed_all(42069)

In [ ]:
data = data_map('jpg')

In [ ]:
train_map, val_map, test_map = train_val_test_split(data)

### Leakage test:

In [ ]:
tr = []
ts = []
vl = []

for cl, samp in train_map.items():
    tr.extend(list(samp))


for cl, samp in test_map.items():
    ts.extend(list(samp))
    

for cl, samp in val_map.items():
    vl.extend(list(samp))
    
len(set(tr) & set(ts)) + len(set(ts) & set(vl)) + len(set(tr) & set(vl))

In [ ]:
from torch import nn

class ImageDataSet(Dataset):
    
    def __init__(self, data_map, data_transforms):
        self.data = data_map
        self.transform = data_transforms
        self._len = sum([len(v) for v in self.data.values()])
        
    def __len__(self):
        return self._len
        
    def __getitem__(self, idx):
        for cls, samples in self.data.items():
            
            if idx < len(samples):
                item = {'X': samples[idx], 'y': cls}
                break
                
            else:
                idx -= len(samples)
                
        else:
            raise IndexError(f"Index {idx} is out of range for dataset with length {self._len}")
        
        image = cv2.imread(item['X'])[:,:,::-1].copy()
        image = self.transform(image)
        label = torch.tensor(item['y'])
        
        return image, label

                                                                                   
                                                                                   
class NLL_OHEM(nn.NLLLoss):                                                     
    """ Online hard example mining. 
    Needs input from nn.LogSotmax() """                                             
                                                                                   
    def __init__(self, ratio, *args, **kwargs):      
        super(NLL_OHEM, self).__init__(*args, **kwargs)                                 
        self.ratio = ratio
                                                                                   
    def forward(self, x, y, ratio=None):                                           
        if ratio is not None:                                                      
            self.ratio = ratio        
            
        num_inst = x.size(0)                                                       
        num_hns = int(self.ratio * num_inst)
        
        x_ = x.clone()    
        inst_losses = torch.autograd.Variable(torch.zeros(num_inst)).cuda()      
        
        for idx, label in enumerate(y.data):                                       
            inst_losses[idx] = -x_.data[idx, label]    
            
        _, idxs = inst_losses.topk(num_hns)                                        
        x_hn = x.index_select(0, idxs)                                             
        y_hn = y.index_select(0, idxs)                                             
        return nn.functional.nll_loss(x_hn, y_hn)
    
class CrossEntropy_OHEM(nn.CrossEntropyLoss):
    
    def __init__(self, ratio, *args, **kwargs):
        super(CrossEntropy_OHEM, self).__init__(*args, **kwargs)                                 
        self.nll_ohem = NLL_OHEM(ratio, *args, **kwargs)
        
    def forward(self, x, y):
        lsm = nn.functional.log_softmax(x, 1)
        return self.nll_ohem(lsm, y)
                                         

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25, unfreeze_epoch=0):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    history = {'loss': {'train': [], 'val': [], 'test': []},
               'acc': {'train': [], 'val': [], 'test': []},}

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        
        val_dset_iter = iter(dataloaders['val'])
        test_dset_iter = iter(dataloaders['test'])
        
        # Each epoch has a training and validation phase
        
        model.train()  # Set model to training mode
        
        # unfreeze
        if epoch == unfreeze_epoch:
            for param in model.parameters():
                param.requires_grad = True
        

        # Train loop
        for iter_num, (inputs, labels) in enumerate(tqdm(dataloaders['train'])):
            inputs = inputs.to(device)
            labels = labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()
            
            if iter_num % (dataset_sizes['train']//dataset_sizes['val'] + 1) == 0:
                # forward
                with torch.no_grad():
                    inp_val, lbl_val = next(val_dset_iter)
                    inp_val = inp_val.to(device)
                    lbl_val = lbl_val.to(device)
                    out_val = model(inp_val)
                    loss_val = criterion(out_val, lbl_val)
                    history['loss']['val'].append(loss_val.item())
                    
                    inp_test, lbl_test = next(test_dset_iter)
                    inp_test = inp_test.to(device)
                    lbl_test = lbl_test.to(device)
                    out_test = model(inp_test)
                    loss_test = criterion(out_test, lbl_test)
                    history['loss']['test'].append(loss_test.item())
            
            optimizer.zero_grad()

            # forward
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # backward + optimize
            loss.backward()
            optimizer.step()

            # statistics
            history['loss']['train'].append(loss.item())
        
        if scheduler:
            scheduler.step(loss)
        
        model.eval()   # Set model to evaluate mode
        
        for mode in ['train', 'val', 'test']:
            
            running_corrects = 0
            for inputs, labels in tqdm(dataloaders[mode]):
                
                inputs = inputs.to(device)
                labels = labels.to(device)

                # forward
                with torch.no_grad():
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)

                running_corrects += torch.sum(preds == labels.data)
                
            epoch_acc = (running_corrects.double() / dataset_sizes[mode]).item()
            history['acc'][mode].append(epoch_acc)

            print('{} Acc: {:.4f}'.format(mode, epoch_acc))

            # deep copy the model
            if mode == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

            print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, history


def eval_model(model, dataloader):
    corrects=0
    model.eval()
    for inputs, labels in tqdm(dataloader):

        inputs = inputs.to(device)
        labels = labels.to(device)

        # forward
        with torch.no_grad():
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

        corrects += torch.sum(preds == labels.data)

    epoch_acc = (corrects.double() / len(dataloader.dataset)).item()
    print(f'Test accuracy: {epoch_acc}')


In [ ]:
save_data = []

for seed in [42069, 1337]:
    seed_all(seed)
    data = data_map('jpg')
    train_map, val_map, test_map = train_val_test_split(data)
    mean_std = get_mean_std(train_map)

    data_transforms = {
        'train': transforms.Compose([
            transforms.ToPILImage(),
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(mean_std['mean'], mean_std['std'])
        ]),

        'val': transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean_std['mean'], mean_std['std'])
        ]),
    }


    image_datasets = {'train': FlowerDSet(train_map, data_transforms['train']),
                      'val': FlowerDSet(val_map, data_transforms['val']),
                      'test': FlowerDSet(test_map, data_transforms['val'])}

    dataloaders = {x: DataLoader(image_datasets[x], batch_size=64, shuffle=(True), num_workers=8)
                  for x in ['train', 'val', 'test']}

    dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val', 'test']}

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    model_ft = models.resnet50(pretrained=True)#, aux_logits=False)
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Linear(num_ftrs, 102)

    model_ft = model_ft.to(device)

    # freeze layers
    for param in model_ft.parameters():
        param.requires_grad = False

    model_ft.fc.requires_grad_(True)

    criterion = CrossEntropy_OHEM(0.3)

    # Observe that all parameters are being optimized
    optimizer_ft = optim.SGD(model_ft.parameters(), lr=1e-3, momentum=0.9)#, nesterov=True)

    # Decay LR by a factor of 0.5 every 5 epochs
    exp_lr_scheduler = lr_scheduler.ReduceLROnPlateau(optimizer_ft, factor=0.5, patience=5,)
    
    # train
    model_ft, hist = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=50, unfreeze_epoch=5)
    model_ft.to(torch.device('cpu'))
    
    save_data.append((model_ft, hist))
    
torch.save(save_data, 'resnet-hist')

In [ ]:
res = torch.load('resnet-hist.zip')
inc = torch.load('inception-hist.zip')

In [ ]:
def smooth(x, f):
    return np.convolve(x, np.ones(f)/f, mode='valid')

def plot_acc(hist, model_name):
    
    plt.figure(figsize=(15, 5))
    for idx, data in enumerate(hist, 1):
        tr = data['acc']['train']
        vl = data['acc']['val']
        ts = data['acc']['test']
        
        plt.plot(tr, label=f'Train Accuracy {model_name} #{idx}')
        plt.plot(vl, label=f'Validation Accuracy {model_name} #{idx}')        
        plt.plot(ts, label=f'Test Accuracy {model_name} #{idx}')
        
        ep = np.argmax(vl)
        print(f"{model_name} test acc. at epoch {ep}: {ts[ep]:.4} (best epoch on val)")
        
    plt.xlabel("Epoch number")
    plt.ylabel("Accuracy")
    plt.title(f"{model_name} Accuracy")
    plt.legend()

    
def plot_losses(hist, model_name):
    w=10
    plt.figure(figsize=(15, 5))
    for idx, data in enumerate(hist, 1):
        tr = smooth(data['loss']['train'], w)
        vl = smooth(np.repeat(data['loss']['val'], 3), w)
        ts = smooth(np.repeat(data['loss']['test'], 3), w)
        
        plt.plot(tr, label=f'Train Loss {model_name} #{idx}')
        plt.plot(vl, label=f'Validation Loss {model_name} #{idx}')        
        plt.plot(ts, label=f'Test Loss {model_name} #{idx}')
        
    
    plt.xlabel("Iteration number")
    plt.ylabel("Loss")
    plt.title(f"{model_name} Loss")
    plt.legend()

In [ ]:
plot_losses(res, 'Resnet')

In [ ]:
plot_acc(res, 'Resnet')

In [ ]:
plot_losses(inc, 'Inception v3')

In [ ]:
plot_acc(inc, 'Inception v3')